In [1]:
import sys
import pandas as pd
from dotenv import load_dotenv
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

# require "HUGGINGFACEHUB_API_TOKEN" in .env file!
load_dotenv()

sys.path.append("../")
from utils.data_utils import preprocess_score_df, preprocess_pattern_map_df
from utils.data_structurer import DataStructurer
from utils.constants import ANSWER_COL

In [2]:
score_df = pd.read_csv("../data/csa_score_1.csv")
pattern_map_df = pd.read_csv("../data/pattern_map.csv")
score_df = preprocess_score_df(score_df)
pattern_map_df = preprocess_pattern_map_df(pattern_map_df)

In [3]:
structurer = DataStructurer(score_df, pattern_map_df, 1122128, 10)

In [4]:
def preprocess(input_string):
    preprocessed_string = input_string.strip()
    preprocessed_string = preprocessed_string.strip("\n")
    return preprocessed_string

In [5]:
template = """Compute the overall score of the response based on the provided criteria. (Max score is {max_score})
###Criteria### 
{criteria}

###Answer### 
{answer}

###Score###
"""

template = preprocess(template)
prompt = PromptTemplate(template=template, input_variables=["criteria", "answer", "max_score"])

In [6]:
# llm = HuggingFaceHub(
#         # repo_id="google/flan-t5-base",
#         # repo_id="google/flan-t5-large",
#         # repo_id="SeaLLMs/SeaLLM-7B-v1",
#         repo_id="scb10x/typhoon-7b",
#         # repo_id="openthaigpt/openthaigpt-1.0.0-alpha-7b-chat-ckpt-hf"
#         # repo_id="migtissera/Tess-M-Creative-v1.0",
#         # repo_id="google/flan-t5-xl",
#         # model_kwargs={
#         #     "temperature": 0,
#         #     "max_length": 128,
#         # }
#     )

# llm_chain = LLMChain(
#     prompt=prompt,
#     llm=llm,
# )

In [7]:
# criteria = """
# + 2 point for having risk mitigation.
# + 3 point for having data-driven decision.
# """
# answer = """Data-Driven Decisions and Risk Mitigation
# """
# max_score = 5
# # criteria = preprocess(criteria)
# # answer = preprocess(answer)

# # result = llm_chain.run({"criteria": criteria, "answer": answer})
# # print(result)


# def predict_score(answer, pattern, max_score):
#     criteria = preprocess(pattern) 
#     answer = preprocess(answer)

#     result = llm_chain.run({"criteria": criteria, "answer": answer, "max_score": max_score})
    
#     try:
#         result = float(result)
#     except ValueError:
#         print(result)
#         result = -1
#     return result

# predict_score(answer, criteria, max_score)

In [8]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="scb10x/typhoon-7b",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 10},
    device=1,
)

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "scb10x/typhoon-7b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

chain = prompt | hf

def predict_score(answer, pattern, max_score):
    criteria = preprocess(pattern) 
    answer = preprocess(answer)

    result = chain.invoke({"criteria": criteria, "answer": answer, "max_score": max_score})
    
    try:
        result = float(result)
    except ValueError:
        print(result)
        result = -1
    return result

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
predict_list = []
for index, row, in structurer.score_df[:20].iterrows():
    pred_score_list = []
    answer = row[ANSWER_COL]
    
    for pattern_idx, (pattern, pattern_max_score) in enumerate(zip(structurer.pattern_list, structurer.pattern_max_score_list)):
        pred_score = predict_score(answer, pattern, pattern_max_score)
        pred_score_list.append(pred_score)
        # print(answer, pattern, pattern_max_score)
        print(f"{index=}, {pattern_idx=}\t, Actual: {row[f'pattern1_{pattern_idx+1}']}, Pred: {pred_score}, Max: {pattern_max_score}")
    predict_list.append(pred_score_list)
        
# structurer.pattern_list
predict_list

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 
The team also achieved the highest score achieved
index=0, pattern_idx=0	, Actual: 0.0, Pred: -1, Max: 2.0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Now the problem was solved. 


index=0, pattern_idx=1	, Actual: 0.0, Pred: -1, Max: 1.0


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 
The 7-8-9-10
index=6, pattern_idx=0	, Actual: 2.0, Pred: -1, Max: 2.0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 
The target rate of growth for 1.
index=6, pattern_idx=1	, Actual: 1.0, Pred: -1, Max: 1.0


KeyboardInterrupt: 